# Variational Autoencoder

- https://github.com/pytorch/examples/blob/master/vae/main.py

- http://cympfh.cc/paper/VAE.html
- https://jaan.io/what-is-variational-autoencoder-vae-tutorial/

In [1]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [95]:
batch_size = 128
epochs = 10
seed = 1
log_interval = 10

cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available!')

In [96]:
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data',
                   train=True,
                   download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

Processing...
Done!


In [10]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data',
                   train=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

In [11]:
print(len(train_loader))
print(len(test_loader))

469
79


In [111]:
class VAE(nn.Module):

    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)  # mu
        self.fc22 = nn.Linear(400, 20)  # logvar
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
#             print('std:', std.size())
            eps = Variable(std.data.new(std.size()).normal_())
#             print('eps:', eps.size())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))
    
    def forward(self, x):
        x = x.view(-1, 784)
        mu, logvar = self.encode(x)
#         print('mu:', mu.size())
#         print('logvar:', logvar.size())
        z = self.reparameterize(mu, logvar)
#         print('z:', z.size())
        return self.decode(z), mu, logvar

- `exp_()` でなくて `exp()` でも同じでは？
- eps（N(0,1) の乱数）がベクトルになっている
- 潜在変数の各次元ごとに異なる乱数をかけることになる
- 数式見るとスカラーみたいだけどこの実装でよいのか？
- `eps = Variable(std.data.new(1).normal_())` ではダメ？
- $logvar = \log \Sigma^2$
- $std = \exp ( 0.5 \log \Sigma^2) = \exp (\log \Sigma) = \Sigma$

In [112]:
model = VAE()
if cuda:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [113]:
data, _ = iter(train_loader).next()
print(data.size())

torch.Size([128, 1, 28, 28])


In [114]:
outputs, mu, logvar = model(Variable(data))
print(outputs.size())
print(mu.size())
print(logvar.size())

torch.Size([128, 784])
torch.Size([128, 20])
torch.Size([128, 20])


In [115]:
def loss_function(recon_x, x, mu, logvar):
    # size_average=Falseなのでバッチ内のサンプルの合計loss
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [116]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        if cuda:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)
            ))
    
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch,
        train_loss / len(train_loader.dataset)
    ))

- VAEは教師なし学習なのでラベルは使わない

In [117]:
for epoch in range(1, epochs + 1):
    train(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 550.498535
Train Epoch: 1 [1280/60000 (2%)]	Loss: 304.871979
Train Epoch: 1 [2560/60000 (4%)]	Loss: 246.235764
Train Epoch: 1 [3840/60000 (6%)]	Loss: 227.771255
Train Epoch: 1 [5120/60000 (9%)]	Loss: 211.217621
Train Epoch: 1 [6400/60000 (11%)]	Loss: 214.487366
Train Epoch: 1 [7680/60000 (13%)]	Loss: 204.445663
Train Epoch: 1 [8960/60000 (15%)]	Loss: 203.452652
Train Epoch: 1 [10240/60000 (17%)]	Loss: 194.351486
Train Epoch: 1 [11520/60000 (19%)]	Loss: 193.790466
Train Epoch: 1 [12800/60000 (21%)]	Loss: 175.084488
Train Epoch: 1 [14080/60000 (23%)]	Loss: 170.913773
Train Epoch: 1 [15360/60000 (26%)]	Loss: 169.013672
Train Epoch: 1 [16640/60000 (28%)]	Loss: 166.865723
Train Epoch: 1 [17920/60000 (30%)]	Loss: 163.366638
Train Epoch: 1 [19200/60000 (32%)]	Loss: 154.371277
Train Epoch: 1 [20480/60000 (34%)]	Loss: 147.533264
Train Epoch: 1 [21760/60000 (36%)]	Loss: 147.393219
Train Epoch: 1 [23040/60000 (38%)]	Loss: 159.925705
Train Epoch: 1 [24320/60000 

KeyboardInterrupt: 